
# Entrenamiento de Red Neuronal Convolucional con ResNet50

Este notebook contiene el código necesario para cargar datos, construir, entrenar y evaluar una red neuronal convolucional para la tarea de regresión de edad utilizando ResNet50.

## Pasos incluidos:
1. **Cargar los datos:** Usaremos un generador de imágenes para manejar los conjuntos de datos.
2. **Crear el modelo:** Utilizaremos ResNet50 con pesos preentrenados en ImageNet.
3. **Entrenar el modelo:** Entrenaremos el modelo y evaluaremos su desempeño con la métrica MAE.


In [ ]:

import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam


In [ ]:

def load_data(path, subset='training'):
    labels = pd.read_csv(path + 'labels.csv')
    
    # Preprocesamiento de imágenes
    data_datagen = ImageDataGenerator(
        rescale=1./255,         # Normalizar los valores de píxeles
        validation_split=0.2    # División en conjunto de validación
    )
    
    data_gen_flow = data_datagen.flow_from_dataframe(
        dataframe=labels,
        directory=path + 'final_files/',  # Directorio de imágenes
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),           # Tamaño esperado por ResNet50
        batch_size=16,
        class_mode='raw',                 # Tarea de regresión
        subset=subset,                    # Subconjunto de entrenamiento o validación
        seed=12345                        # Reproducibilidad
    )

    return data_gen_flow


In [ ]:

def create_model(input_shape):
    # Base de ResNet50 con pesos preentrenados
    backbone = ResNet50(weights='imagenet', 
                        input_shape=input_shape, 
                        include_top=False)

    # Crear el modelo completo
    model = Sequential()
    model.add(backbone)  # Modelo base
    model.add(GlobalAveragePooling2D())  # Reducción de dimensionalidad
    model.add(Dense(128, activation='relu'))  # Capa intermedia
    model.add(Dropout(0.5))  # Regularización
    model.add(Dense(1, activation='relu'))  # Salida para la regresión

    # Compilación del modelo
    optimizer = Adam(learning_rate=0.0005)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

    return model


In [ ]:

def train_model(model, train_data, test_data, batch_size=None, epochs=3):
    # Entrenamiento del modelo
    model.fit(
        train_data, 
        validation_data=test_data, 
        batch_size=batch_size, 
        epochs=epochs,
        verbose=2
    )
    return model


In [ ]:

# Configuración inicial
input_shape = (224, 224, 3)  # Formato de las imágenes
train = load_data('/datasets/train/')  # Cargar datos de entrenamiento
test = load_data('/datasets/test/', subset='validation')  # Cargar datos de validación

# Crear y entrenar el modelo
model = create_model(input_shape)
model = train_model(model, train, test, batch_size=16, epochs=3)

# Guardar el modelo entrenado
model.save('age_prediction_model.h5')
